** What I want to do: ** Process test files for distracted driver and submit to kaggle

## Admin stuff

In [1]:
%matplotlib inline

In [2]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
from datetime import datetime
import re

In [3]:
data_dir = "data"
train_path = "data/train/"
test_path = "data/test/"
validation_path = "data/valid/"
sample_train_path = "data/sample/train/"
sample_validation_path = "data/sample/valid/"
results_path = "data/results/"

In [4]:
# some useful utilities and the vgg16 model
import utils; reload(utils)
from utils import plots
from utils import save_array, load_array

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


In [5]:
# import the vgg16 model
import vgg16; reload(vgg16)
from vgg16 import Vgg16, image

## Load model weights


In [6]:
# instantiate the vgg16 model
vgg = Vgg16()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


In [7]:
vgg.model.load_weights(results_path+'ft_1epoch.h5')

## Run model against test set

In [9]:
# for the vgg.test method to run, all images need to be part of another class.
# Lets call it 'unknown'
len(os.listdir(test_path))

79726

In [13]:
tempdir = test_path+'unknown'
!mkdir $tempdir

In [21]:
img_mv_counter = 0
for img in os.listdir(test_path):
    if os.path.isfile(os.path.join(test_path,img)):
        os.rename(os.path.join(test_path,img),os.path.join(test_path,'unknown',img))
        img_mv_counter+=1
print ("{} images moved to unknown folder.".format(str(img_mv_counter)))

17962 images moved to unknown folder.


In [22]:
# start logging time to execute
starttime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

test_batches, test_proba = vgg.test(test_path, batch_size = 64)

# log time for when the process ends
endtime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print ("timedelta = ", datetime.strptime(endtime,'%Y-%m-%d %H:%M:%S') - datetime.strptime(starttime,'%Y-%m-%d %H:%M:%S'))

Found 79726 images belonging to 1 classes.
timedelta =  0:32:13


## Build submission from predicted probabilities

In [23]:
# get filenames for each validation set
filenames = test_batches.filenames
filenames[:10]

['unknown/img_1.jpg',
 'unknown/img_10.jpg',
 'unknown/img_100.jpg',
 'unknown/img_1000.jpg',
 'unknown/img_100000.jpg',
 'unknown/img_100001.jpg',
 'unknown/img_100002.jpg',
 'unknown/img_100003.jpg',
 'unknown/img_100004.jpg',
 'unknown/img_100005.jpg']

In [25]:
len(filenames)

79726

In [26]:
test_proba.shape

(79726, 10)

In [27]:
# taking a look at the first 2...
test_proba[0:2,:]

array([[  6.0330e-03,   6.2223e-03,   2.8167e-03,   3.5212e-04,   6.0967e-04,   1.8071e-01,
          5.2517e-06,   6.0826e-04,   7.3298e-01,   6.9665e-02],
       [  9.9047e-01,   1.2519e-04,   1.5722e-05,   4.6743e-06,   4.2948e-03,   1.7472e-03,
          2.7635e-04,   3.4640e-06,   9.6088e-05,   2.9628e-03]], dtype=float32)

In [28]:
save_array(results_path + 'test_preds.dat', test_proba)
save_array(results_path + 'filenames.dat', filenames)